# Цель
***Предсказание оттока клиентов для KKBOX***

In [2]:
from __future__ import division
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

    # Импортируем и просмотрим таблицу transactions
dataTableTransaction = pd.read_csv('transactions_v.csv', error_bad_lines=False)
dataTableTransaction.head()

b'Skipping line 26005: expected 9 fields, saw 14\nSkipping line 39009: expected 9 fields, saw 14\n'
b'Skipping line 4239114: expected 9 fields, saw 13\n'


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32.0,90.0,298.0,298.0,0.0,20170131.0,20170504.0,0.0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41.0,30.0,149.0,149.0,1.0,20150809.0,20190412.0,0.0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36.0,30.0,180.0,180.0,1.0,20170303.0,20170422.0,0.0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36.0,30.0,180.0,180.0,1.0,20170329.0,20170331.0,1.0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41.0,30.0,99.0,99.0,1.0,20170323.0,20170423.0,0.0


In [3]:
    # Загружаем данные из sample_submission и train
dataTableSampleSubmission = pd.read_csv('sample_submission_v.csv')
dataTableTrain = pd.read_csv('train_v.csv')

In [4]:
    # собираем в dataTableIsChurn целевое значение из таблицы train
dataTableIsChurn = [dataTableSampleSubmission,dataTableTrain]
dataTableIsChurn = pd.concat(dataTableIsChurn)
dataTableIsChurn['is_churn'].sum()

87330

In [ ]:
    # выгружаем данные из user_logs
dataTableLog = pd.read_csv('user_logs_v.csv')
dataTableLog.head()

In [5]:
    # выгружаем данные из members и объединяем их с предыдущей
dataTableMember = pd.read_csv('members.csv')
dataTableMember.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [6]:
# объединяем таблицу в общую
dataTableMerge = pd.merge(dataTableMember, dataTableIsChurn, how= 'left', on = 'msno')
dataTableMerge = pd.merge(dataTableMerge, dataTableTransaction, how= 'left', on = 'msno')
dataTableMerge.head()

,msno,city,bd,gender,registered_via,registration_init_time,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,0.0,41.0,30.0,129.0,129.0,1.0,20170327.0,20170428.0,0.0
2,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,0.0,41.0,30.0,129.0,129.0,1.0,20170327.0,20170428.0,0.0
3,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,0.0,41.0,30.0,129.0,129.0,1.0,20170327.0,20170428.0,0.0
4,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,0.0,41.0,30.0,129.0,129.0,1.0,20170327.0,20170428.0,0.0


In [7]:
    # предобработка данных : удаляем повторения, удаляем пустые значения
dataTableMerge = dataTableMerge.drop_duplicates()
dataTableMerge = dataTableMerge.dropna()
dataTableMerge.head()

,msno,city,bd,gender,registered_via,registration_init_time,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
9,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915,0.0,40.0,30.0,149.0,149.0,1.0,20170314.0,20170413.0,0.0
12,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916,0.0,39.0,30.0,149.0,149.0,1.0,20170331.0,20170517.0,0.0
15,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916,0.0,39.0,30.0,149.0,149.0,1.0,20170228.0,20170417.0,0.0
23,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918,0.0,40.0,30.0,149.0,149.0,1.0,20170327.0,20170426.0,0.0
37,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,28,female,9,20110920,0.0,36.0,30.0,180.0,180.0,1.0,20170303.0,20170402.0,0.0


In [8]:
    # Все строковые атрибуты были преобразованы в числовые
dataTableMerge['sex'] = np.where(dataTableMerge['gender'] == 'male',1,0)
dataTableMerge[['is_churn']] = dataTableMerge[['is_churn']].astype(int)
print(dataTableMerge.dtypes)

msno                       object
city                        int64
bd                          int64
gender                     object
registered_via              int64
registration_init_time      int64
is_churn                  float64
payment_method_id         float64
payment_plan_days         float64
plan_list_price           float64
actual_amount_paid        float64
is_auto_renew             float64
transaction_date          float64
membership_expire_date    float64
is_cancel                 float64
sex                         int32
dtype: object


In [9]:
    # выносим целевую переменную отдельно
    # удаляем неиспользуемые колонки
y = dataTableMerge['is_churn']
сolsForDrop = ['city','bd','registered_via','msno', 'gender','registration_init_time',
             'transaction_date','membership_expire_date','is_churn','is_auto_renew','is_cancel','sex']
dataTableMergeClean = dataTableMerge.drop(сolsForDrop,axis=1)
dataTableMergeClean = dataTableMergeClean.dropna() 
dataTableMergeClean.head()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid
9,40.0,30.0,149.0,149.0
12,39.0,30.0,149.0,149.0
15,39.0,30.0,149.0,149.0
23,40.0,30.0,149.0,149.0
37,36.0,30.0,180.0,180.0


In [10]:
# обновляем индексы, и смотрим содержится ли null в каком нибудь столбце
dataTableMergeClean = dataTableMergeClean.reset_index()
print(dataTableMergeClean.isnull().any())
dataTableMergeClean = dataTableMergeClean.fillna(method='ffill')
print('---')
dataTableMergeClean.head()

index                 False
payment_method_id     False
payment_plan_days     False
plan_list_price       False
actual_amount_paid    False
dtype: bool
---


,index,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid
0,9,40.0,30.0,149.0,149.0
1,12,39.0,30.0,149.0,149.0
2,15,39.0,30.0,149.0,149.0
3,23,40.0,30.0,149.0,149.0
4,37,36.0,30.0,180.0,180.0


In [11]:
# Заполняем таблицу features используемыми фичами
features = dataTableMergeClean.columns
print(features)

Index(['index', 'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid'],
      dtype='object')


In [12]:
# определяем X
X = dataTableMergeClean.as_matrix().astype(np.float32)

#Стандартизируем значения    
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import Imputer

scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
X = imp.fit_transform(X)


print('Пространство объектов содержит% d наблюдений и% d фич' % X.shape)
print(X.dtype)

Пространство объектов содержит 536590 наблюдений и 5 фич
float32


In [260]:
def is_finite(df):
    print(df[:,0].shape)
    index = 0
    for i in df[:,0]:
        if not np.isfinite(i):
            print(index, i)
        index +=1
is_finite(X)
print('---')

def _assert_all_finite(X):
    X = np.asanyarray(X)
    if (X.dtype.char in np.typecodes['AllFloat'] and not np.isfinite(X.sum())
            and not np.isfinite(X).all()):
        raise ValueError("Вводимое значение содержит NaN, бесконечность"
                         " или слишком большое %r." % X.dtype)
print(_assert_all_finite(X))
print('---')
print(np.any(np.isnan(X)))
print('---')
print(np.all(np.isfinite(X)))

(536590,)
---
None
---
False
---
True


In [ ]:
# очищаем данные от nan, Inf или пропущенные значения
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float32)

clean_dataset(dataTableMergeClean)
print(dataTableMergeClean.dtypes)

In [ ]:
from sklearn.cross_validation import KFold

def run_cross_validation(X,y,clf_class,**kwargs):
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_pred=y.copy()
    
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(x_test)
        return y_pred

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

def accuracy(y_true,y_pred):
    return np.mean(y_true == y_pred)

print("svm:")
print("%.3f" % accuracy(y, run_cross_validation(X,y,SVC)))
print("rf:")
print("%.3f" % accuracy(y, run_cross_validation(X,y,RF)))
print("knn:")
print("%.3f" % accuracy(y, run_cross_validation(X,y,KNN)))

реализация svm

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
clf = SVC()
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [17]:
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

y_pred = clf.predict(X_test)

accurate_score = metrics.accuracy_score(y_test, y_pred) 
print(accurate_score)
print('---')

0.7645727911441
---
